<H3>HPDP ASSIGNMENT 2<H3>
<p>Members:<p>
<p>LIM JING YONG A22EC0182</p>
<p>LEE SOON DER A22EC0065</p>

In [11]:
from google.colab import files
files.upload()

{}

In [ ]:
import os
import shutil

# Create the Kaggle folder if it doesn't exist
os.makedirs("/root/.kaggle", exist_ok=True)

# Move the uploaded kaggle.json to the right location
shutil.move("kaggle.json", "/root/.kaggle/kaggle.json")

# Set the correct permissions
os.chmod("/root/.kaggle/kaggle.json", 600)

In [ ]:
!pip install -q kaggle

In [ ]:
# Download the dataset using Kaggle API
!kaggle datasets download -d tavoglc/covid19-in-the-american-continent

Dataset URL: https://www.kaggle.com/datasets/tavoglc/covid19-in-the-american-continent
License(s): other
 94% 499M/531M [00:01<00:00, 256MB/s]
100% 531M/531M [00:01<00:00, 340MB/s]


In [ ]:
# Unzip the downloaded file
!unzip -q covid19-in-the-american-continent.zip

In [1]:
import pandas as pd
import time
import psutil
import os

process = psutil.Process(os.getpid())
mem_before = process.memory_info().rss / 1024 ** 2  # in MB

start_time = time.time()

# Load one of the CSV files
df = pd.read_csv("continental2.csv")

mem_after = process.memory_info().rss / 1024 ** 2  # in MB
end_time = time.time()

# Preview the data
print(df.head())

print("Performance Measurement without Optimization Techniques")
print(f"Execution Time: {end_time - start_time:.2f} seconds")
print(f"Memory Used: {mem_after - mem_before:.2f} MB")

   Unnamed: 0.1  Unnamed: 0        date  cases country  \
0             0           0  2020-02-19    1.0  mexico   
1             1           1  2020-02-22    1.0  mexico   
2             2           2  2020-02-22    1.0  mexico   
3             3           3  2020-02-23    1.0  mexico   
4             4           4  2020-02-25    1.0  mexico   

                                                 qry        lat       long  \
0                lat==19.5600234 & long==-99.1312722  19.560023 -99.131272   
1                  lat==19.359004 & long==-99.092623  19.359004 -99.092623   
2           lat==20.093813 & long==-98.7923761142857  20.093813 -98.792376   
3                  lat==19.482945 & long==-99.113471  19.482945 -99.113471   
4  lat==16.70864529166667 & long==-93.13649273214284  16.708645 -93.136493   

   dayofyear  year  lengthofday     delta    delta2  normilized_cases  
0         50  2020    12.274737  0.015820  0.000164               0.0  
1         53  2020    12.328092  0.016

Big Data Handling Startegies with Dask(64MB):

1. Load less dataset
2. Optimize data types
3. Chunking dataset with dask
4. Sampling


In [2]:
import dask.dataframe as dd
import time
import psutil
import os

op_process = psutil.Process(os.getpid())
op_mem_before = op_process.memory_info().rss / 1024 ** 2  # in MB
op_start_time = time.time()

#1 Load less dataset

columns_to_use = ['date', 'cases', 'country', 'lat', 'long']

#2 Optimize data types

optimized_dtypes = {
    'cases': 'float32',
    'lat': 'float32',
    'long': 'float32',
    'country': 'category'
}

#3 Read CSV using dask with chunking
df64_dask = dd.read_csv(
    "continental2.csv",
    usecols=columns_to_use,
    dtype=optimized_dtypes,
    parse_dates=['date'],
    blocksize="64MB"  #Controls chunk/partition size
)

#4 Sampling
df64_dask_sampled = df64_dask.sample(frac=0.1, random_state=42)

#Trigger Cmomputation
df64_optimized = df64_dask_sampled.compute()

op_mem_after = op_process.memory_info().rss / 1024 ** 2  # in MB
op_end_time = time.time()

# Inspect result
print(df64_optimized.info())
print(df64_optimized.head())

print("Dask partitions:", df64_dask.npartitions)

print("Performance Measurement with Big Data Handling Strategies")
print(f"Execution Time: {op_end_time - op_start_time:.2f} seconds")
print(f"Memory Used: {op_mem_after - op_mem_before:.2f} MB")

<class 'pandas.core.frame.DataFrame'>
Index: 616489 entries, 327154 to 200459
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   date     616489 non-null  datetime64[ns]
 1   cases    616489 non-null  float32       
 2   country  616489 non-null  category      
 3   lat      616489 non-null  float32       
 4   long     616489 non-null  float32       
dtypes: category(1), datetime64[ns](1), float32(3)
memory usage: 17.1 MB
None
             date  cases country        lat        long
327154 2021-09-08   42.0  mexico  19.318882 -103.775024
111548 2020-10-08    1.0  mexico  20.665031  -89.660858
120356 2020-10-23    1.0  mexico  20.661798  -99.892487
170425 2021-01-07    6.0  mexico  17.031370  -96.696899
129586 2020-11-08    1.0  mexico  20.232145  -98.545509
Dask partitions: 16
Performance Measurement with Big Data Handling Strategies
Execution Time: 44.32 seconds
Memory Used: 50.72 MB


Big Data Handling Strategies with Dask (Optimization with 128MB)

In [4]:
import dask.dataframe as dd
import time
import psutil
import os
from dask.distributed import Client

dop_process = psutil.Process(os.getpid())
dop_mem_before = dop_process.memory_info().rss / 1024 ** 2  # in MB
dop_start_time = time.time()

# Optional: Start Dask distributed client (for diagnostics & speed)
# client = Client(processes=True, n_workers=4, threads_per_worker=2)

#1 Load less dataset

columns_to_use = ['date', 'cases', 'country', 'lat', 'long']

#2 Optimize data types

optimized_dtypes = {
    'cases': 'float32',
    'lat': 'float32',
    'long': 'float32',
    'country': 'category'
}

#3 Read CSV using dask with chunking
df128_dask = dd.read_csv(
    "continental2.csv",
    usecols=columns_to_use,
    dtype=optimized_dtypes,
    parse_dates=['date'],
    blocksize="128MB"  #Controls chunk/partition size
)

#4 Sampling
df128_dask_sampled = df128_dask.sample(frac=0.1, random_state=42)

# Persist in memory (reduces recomputation)
# df_dask_sampled = df_dask_sampled.persist()

#Trigger Cmomputation
df128_optimized = df128_dask_sampled.compute()

dop_mem_after = dop_process.memory_info().rss / 1024 ** 2  # in MB
dop_end_time = time.time()

# Inspect result
print(df128_optimized.info())
print(df128_optimized.head())

print("Dask partitions:", df128_dask.npartitions)

print("Performance Measurement with Big Data Handling Strategies")
print(f"Execution Time: {dop_end_time - dop_start_time:.2f} seconds")
print(f"Memory Used: {dop_mem_after - dop_mem_before:.2f} MB")

<class 'pandas.core.frame.DataFrame'>
Index: 616489 entries, 333763 to 600551
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   date     616489 non-null  datetime64[ns]
 1   cases    616489 non-null  float32       
 2   country  616489 non-null  category      
 3   lat      616489 non-null  float32       
 4   long     616489 non-null  float32       
dtypes: category(1), datetime64[ns](1), float32(3)
memory usage: 17.1 MB
None
             date  cases country        lat        long
333763 2021-09-15    6.0  mexico  23.221594 -102.972984
687536 2022-04-07   12.0     usa  39.627544 -106.693367
675906 2020-05-12   11.0     usa  40.093136 -105.354675
613124 2021-02-14    1.0     usa  35.022018  -90.748283
258990 2021-06-25    1.0  mexico  14.751727  -92.207458
Dask partitions: 8
Performance Measurement with Big Data Handling Strategies
Execution Time: 42.55 seconds
Memory Used: 13.61 MB
